In [8]:
import os
from llama_index.core import SimpleDirectoryReader

In [6]:
file_dir = "../sample_data/"
reader = SimpleDirectoryReader(input_dir=file_dir)
documents = reader.load_data()

In [5]:
print(len(documents))

28


In [9]:
os.listdir(file_dir)

['2402.05602v1.pdf', 'd20e3b9a-9981-41c4-a6f7-dc1bafff4761.JPG']

In [10]:
documents[0]

Document(id_='adb5bb59-c7af-4bbf-9d2d-788d008ed952', embedding=None, metadata={'page_label': '1', 'file_name': '2402.05602v1.pdf', 'file_path': '/Users/ashwin/side_projects/llama-fs/notebooks/../sample_data/2402.05602v1.pdf', 'file_type': 'application/pdf', 'file_size': 14982917, 'creation_date': '2024-05-11', 'last_modified_date': '2024-05-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='AttnLRP: Attention-Aware Layer-wise Relevance Propagation\nfor Transformers\nReduan Achtibat1Sayed Mohammad Vakilzadeh Hatefi1Maximilian Dreyer1\nAakriti Jain1Thomas Wiegand1,2,3Sebastian Lapuschkin1,†Wojciech Samek1,2,3,†\n1Fraunhofer Heinrich-Hertz-Institute, 10587 Berlin, Germany\n2Technische Universit ¨at Berlin, 10587 Berlin, Germany\n3BIFOLD – Berlin Institut